In [1]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt 
from skimage import io, filters
# from skimage.restoration import denoise_nl_means, estimate_sigma
import os
from glob import glob

# Methods
from utilities import background_correction

In [2]:
# Define the condition
condition = 'Flat_after_bursting'

dates = sorted(os.listdir(f'Data/Pourcentage/{condition}'))
for date in dates:
    integrins = sorted(glob(f'Data/Pourcentage/{condition}/{date}/Ig*.tif'))
    talins = sorted(glob(f'Data/Pourcentage/{condition}/{date}/T*.tif'))
    for integrin, talin in zip(integrins, talins):
        print(date, integrin[-7:-4], talin[-6:-4])

20201202 Ig1 T1
20201202 Ig2 T2
20201202 Ig3 T3
20201202 Ig4 T4
20201202 Ig5 T5
20201202 Ig6 T6
20201202 Ig7 T7
20201202 Ig8 T8
20201203 Ig1 T1
20201203 Ig2 T2
20201203 Ig3 T3
20201203 Ig5 T5
20201203 Ig6 T6
20201203 Ig7 T7
20201203 Ig8 T8
20201203 Ig9 T9


In [3]:
# List the experiments dates
dates = sorted(os.listdir(f'Data/Pourcentage/{condition}'))
for date in dates:
    # list images for each experiment
    integrins = sorted(glob(f'Data/Pourcentage/{condition}/{date}/Ig*.tif'))
    talins    = sorted(glob(f'Data/Pourcentage/{condition}/{date}/T*.tif' ))
    for integrin, talin in zip(integrins, talins):
        # Read the images
        img_talin    = io.imread(talin)
        img_integrin = io.imread(integrin)
        
        # Read the frame
        frame = integrin[-5]
        
        # Correct the background
        img_talin_corrected = background_correction(img_talin)
        img_integrin_corrected = background_correction(img_integrin)
        
        # # Denoise the image
        # img_talin_corrected = denoise_nl_means(img_talin_corrected, h = np.mean(estimate_sigma(img_talin_corrected)), fast_mode = True, patch_size = 10, patch_distance = 1)
        # img_integrin_corrected = denoise_nl_means(img_integrin_corrected, h = np.mean(estimate_sigma(img_integrin_corrected)), fast_mode = True, patch_size = 10, patch_distance = 1)
        
        # Threshold using otsu
        img_talin_th = img_talin_corrected * (img_talin_corrected > filters.threshold_otsu(img_talin_corrected))
        img_integrin_th = img_integrin_corrected * (img_integrin_corrected > filters.threshold_otsu(img_integrin_corrected))

        # Compute the intersection
        img_i = img_talin_th * img_integrin_th
        img_i[img_i > 0] = 1

        # Compute the union
        img_u = img_talin_th + img_integrin_th
        img_u[img_u > 0] = 1
        
        # Compute the intersection over union
        iou = np.count_nonzero(img_i) / np.count_nonzero(img_u)
        
        # Draw a scale bar
        img_talin[940:960,840:980] = img_talin.max()
        img_integrin[940:960,840:980] = img_integrin.max()
        # plot the figures
        fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(12, 7), tight_layout=True)
        fig.suptitle(f'{condition} {date}-{frame} IOU = {iou:.4f}')

        # fig.set_cmap('hot')
        for ax in axs.ravel():
            ax.set_axis_off()

        # Show the images
        axs[0,0].imshow(img_integrin, cmap='gray')
        axs[1,0].imshow(img_talin, cmap='gray')
        axs[0,1].imshow(img_integrin_corrected, cmap='gray')
        axs[1,1].imshow(img_talin_corrected, cmap='gray')
        axs[0,2].imshow(img_integrin_th, cmap='gray')
        axs[1,2].imshow(img_talin_th, cmap='gray')
        axs[0,3].imshow(img_i, cmap='gray')
        axs[1,3].imshow(img_u, cmap='gray')

        axs[0,0].set_title('Integrin original')
        axs[1,0].set_title('Talin original')
        axs[0,1].set_title('Integrin corrected')
        axs[1,1].set_title('Talin corrected')
        axs[0,2].set_title('Integrin thresholded')
        axs[1,2].set_title('Talin thresholded')
        axs[0,3].set_title('Intersection')
        axs[1,3].set_title('Union')

        plt.savefig(f'Data/Pourcentage/results/{condition}_{date}_{frame}')
        plt.close();